# 고객 지원 멀티 에이전트 서비스

에이전트의 성과를 더욱 향상시키는 데 도움이 되는 6가지 핵심 요소:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

패키지 설치

In [31]:
%pip install -q crewai crewai_tools langchain_community pip setuptools wheel

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Crew, Agent, Task, Process

In [3]:
import os
import getpass
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

True

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="선임 고객 지원 담당자",
    goal="팀에서 가장 친절하고 도움이 되는 고객 지원 담당자가 되세요.",
    backstory=(
        "당신은 Anthropic(https://www.anthropic.com/)에서 근무하고 있으며, "
        "귀사의 매우 중요한 고객인 {customer}에게 지원을 제공하기 위해 노력하고 있습니다."
        "최고의 지원을 제공해야 합니다!"
        "완전한 답변을 제공하고, 어떠한 가정도 하지 마십시오."
    ),
    allow_delegation=False,
    verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [5]:
support_quality_assurance_agent = Agent(
	role="품질 보증 전문가 지원",
	goal="팀에서 최고의 지원 품질 보증을 제공하는 것으로 인정받기",
	backstory=(
		"당신은 Anthropic(https://www.anthropic.com/)에서  근무하고 있으며, "
        "현재 {customer}의 요청에 대해 팀과 협력하여 "
		"지원 담당자가 가능한 최상의 지원을 "
        "제공하고 있는지 확인하고 있습니다. \n"
		"지원 담당자가 완전한 답변을 제공하고 있는지 확인해야 하며, "
        "어떠한 가정도 하지 않아야 합니다. "
	),
	verbose=True
)

* **Role Playing**: 두 에이전트 모두 역할, 목표 및 배경 스토리를 받았습니다.
* **Focus**: 두 에이전트 모두 자신이 연기하는 역할의 캐릭터에 몰입하도록 촉구되었습니다.
* **Cooperation**: 지원 품질 보증 에이전트는 작업을 지원 에이전트에게 위임하여 이러한 에이전트가 함께 작업할 수 있도록 할 수 있습니다.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [7]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### 가능한 사용자 정의 도구
- 고객 데이터 로드
- 이전 대화 활용
- CRM에서 데이터 로드
- 기존 버그 보고서 확인
- 기존 기능 요청 확인
- 진행 중인 티켓 확인
- 기타

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- 문서 스크래퍼 도구를 인스턴스화합니다.
- 이 도구는 CrewAI 문서의 한 페이지(URL 1개만)를 스크래핑합니다.

In [8]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://modelcontextprotocol.io/introduction"
)

##### 에이전트에게 도구를 제공하는 방법

- Agent 레벨: 에이전트는 수행하는 모든 작업에 도구를 사용할 수 있습니다.
- Task 레벨: 에이전트는 해당 특정 작업을 수행할 때만 도구를 사용합니다.

**참고**: Task 도구는 Agent 도구를 재정의(override) 합니다.

### Task 생성

In [9]:
inquiry_resolution = Task(
    description=(
        "{customer}가 방금 매우 중요한 질문을 했습니다.\n"
	    "{inquiry}\n\n"
        "{customer}의 {person}이 연락했습니다. "
		"가능한 한 최상의 지원을 제공하기 위해 아는 모든 것을 활용하세요."
		"고객의 문의에 완전하고 정확한 답변을 제공하기 위해 노력해야 합니다."
    ),
    expected_output=(
	    "고객의 문의에 대한 자세하고 유익한 답변으로,  "
        "질문의 모든 측면을 다룹니다.\n "
        "답변에는 외부 데이터나 솔루션을 포함하여 "
        "답변을 찾는 데 사용한 모든 것에 대한 "
        "참조가 포함되어야 합니다. "
        "답변이 완전하고, "
		"답변하지 않은 질문이 없도록 하고, "
		"처음부터 끝까지 도움이 되고 친절한 답변을 유지하십시오."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` 는 어떤 도구도 사용하지 않습니다.
- QA 에이전트는 지원 에이전트의 작업만 검토합니다.

In [10]:
quality_assurance_review = Task(
    description=(
        "{customer}의 문의에 대한 선임 고객 지원 담당자가 작성한 답변을 검토합니다. "
        "답변이 포괄적이고 정확하며 고객 지원에 기대되는 고품질 표준을 준수하는지 확인합니다.\n"
        "고객 문의의 모든 부분이 도움이 되고 친절한 어조로 철저히 처리되었는지 확인합니다.\n"
        "정보를 찾는 데 사용된 참조 자료와 출처를 확인하여 "
        "답변이 잘 뒷받침되고 답변하지 않은 질문이 없는지 확인합니다."
    ),
    expected_output=(
        "고객에게 보낼 수 있는 상세하고 유익한 최종 답변입니다.\n"
        "이 답변은 고객의 문의 사항을 완전히 다루고 "
        "모든 관련 피드백과 개선 사항을 통합해야 합니다.\n"
		"우리 회사는 차갑고(chill) 쿨(cool)한 회사지만, 너무 격식을 차리지 말고 전반적으로 전문적이고 친근한 톤(tone)을 유지하세요."
    ),
    agent=support_quality_assurance_agent,
)

### Crew 생성

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.
- crew를 구성할 때 `memory=True`를 설정하면 메모리가 활성화됩니다.

In [11]:
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    # process=Process.sequential,
    # memory=True,
    # verbose=True
)

### Crew 실행

**참고**: LLM은 동일한 입력에 대해 다른 출력을 제공합니다.

#### 가드레일
- 아래 실행을 실행하면 에이전트와 응답이 기대하는 범위 내에 있음을 알 수 있습니다.

In [12]:
inputs = {
    "customer": "ABC컴퍼니",
    "person": "김민규",
    "inquiry": "AI 에이전트 시스템 구축을 하는 데 도움이 필요합니다.  "
               "여러 시스템에 AI 에이전트 기술을 활용하는 추가하는 방법은 무엇입니까?  "
               "방법을 알려 주시거나 솔루션을 소개해 주세요."
}
result = crew.kickoff(inputs=inputs)

# Agent: 선임 고객 지원 담당자
## Task: ABC컴퍼니가 방금 매우 중요한 질문을 했습니다.
AI 에이전트 시스템 구축을 하는 데 도움이 필요합니다.  여러 시스템에 AI 에이전트 기술을 활용하는 추가하는 방법은 무엇입니까?  방법을 알려 주시거나 솔루션을 소개해 주세요.

ABC컴퍼니의 김민규이 연락했습니다. 가능한 한 최상의 지원을 제공하기 위해 아는 모든 것을 활용하세요.고객의 문의에 완전하고 정확한 답변을 제공하기 위해 노력해야 합니다.


# Agent: 선임 고객 지원 담당자
## Thought: Thought: I need to find detailed information about integrating AI agent technology into various systems for ABC Company.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Introduction - Model Context Protocol Model Context Protocol home page Search... Python SDK TypeScript SDK Java SDK Kotlin SDK C# SDK Get Started Introduction Quickstart Example Servers Example Clients Tutorials Building MCP with LLMs Debugging Inspector Concepts Core architecture Resources Prompts Tools Sampling Roots Transports Development What's New Roadmap Contributing Model Context Protocol home page Search... GitHub GitHub Search... Navigation Get Started Introduction User Guide SDKs Specification Us

- Display the final result as Markdown.

In [14]:
from IPython.display import Markdown
Markdown(result.raw)

Hello ABC Company Team,

Thank you for your inquiry regarding the implementation of AI agent technology using the Model Context Protocol (MCP). We appreciate your enthusiasm to leverage this technology, and we're here to provide you with detailed insights to ensure a comprehensive understanding.

**Understanding Model Context Protocol (MCP)**:  
MCP is an open protocol designed to standardize how applications offer context to Large Language Models (LLMs). Think of it as a USB-C port for AI applications – it enables smooth integration across various data sources and tools for better performance.

**Benefits of Using MCP**:
- **Seamless Integration**: MCP simplifies creating AI agents and complex workflows leveraging LLMs.
- **Pre-built Integrations**: There’s a growing library of pre-built integrations, allowing LLMs to tap into various functionalities effortlessly.
- **Flexibility & Adaptability**: Switching between different LLM providers is straightforward, helping you quickly adapt to any changing business needs.
- **Data Security Best Practices**: MCP provides direction on how to secure your data throughout the infrastructure.

**General Architecture**:
MCP operates on a client-server architecture, which means that a host application can connect to multiple servers. Here’s what that entails:
- **MCP Hosts**: These are applications (like desktop clients or AI tools) that need access to data.
- **MCP Clients**: These clients maintain connections with specific servers.
- **MCP Servers**: Lightweight programs that expose capabilities through the protocol.
- **Local Data Sources**: These include files, databases, or services on your computer that MCP servers can secure access to.
- **Remote Services**: External systems are accessed via APIs over the internet.

**Getting Started with MCP**:
Depending on your specific needs, you have several approaches to consider:
- **For Server Developers**: Learn about building an MCP server tailored for desktop and client applications.
- **For Client Developers**: Start building a client capable of interfacing with various MCP servers.
- **For Users**: Access pre-built servers in platforms like Claude for Desktop to kickstart your journey.

**Examples and Tutorials**:
You can explore a gallery of official MCP servers and their implementations. In addition, there are tutorials available to guide you through developing your own MCP applications. The MCP documentation, debugging guide, and inspector tools are also at your disposal to support this process.

**Core Concepts**:
- **Resources**: Serve as a bridge to expose data and content from servers to LLMs.
- **Prompts**: Create reusable workflows and templates, enhancing efficiency.
- **Tools**: Equip LLMs with capabilities to perform actions through the servers, thereby amplifying functionality.
- **Sampling and Transports**: Facilitate servers' requests for completions and enhance communication mechanisms within MCP.

**Support and Feedback**:
Should you encounter any issues or have feature enhancement requests, various channels are available for bug reporting. We also recommend joining community discussions concerning MCP for collaborative problem-solving.

By following these guidelines and harnessing the capabilities of MCP, ABC Company can successfully implement AI agent technology across your systems, enhancing efficiency and productivity.

If you have any further questions or need clarification on specific points, please feel free to reach out. We're here to help and ensure your success with MCP.

Best regards,  
[Your Name]  
[Your Position]  
Anthropic